In [24]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from huggingface_hub import hf_hub_download
from gluonts.dataset.common import ListDataset
from pandas.tseries.offsets import DateOffset

from uni2ts.eval_util.plot import plot_single
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
import numpy as np
import plotly.graph_objs as go


SIZE = "base"  # model size: choose from {'small', 'base', 'large'}


In [25]:
# Import the data 
def load_and_prepare_data(file_path):
    """
    Load energy prices data from a CSV file, ensure chronological order, and convert 'Date' to datetime.
    """
    df = pd.read_csv(file_path)
    df.sort_values('Date', inplace=True)
    df.set_index('Date', inplace=True)
    df = pd.DataFrame(df)
    return df

In [26]:
# Import the data
df = load_and_prepare_data('../../data/Final_data/final_data.csv')

# Reset the index
df = df.reset_index()

In [27]:
# Convert 'date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Set the 'date' column as the index
df.set_index('Date', inplace=True)

# Ensure the DataFrame is sorted by the index
df.sort_index(inplace=True)

# setting daily frequency
df = df.asfreq('D')


In [28]:
# Checking the frequency of the dataset
print("Original Frequency:", df.index.freq)

# If the frequency is None or not what you expect, you can set it:
if df.index.freq is None:
    df = df.asfreq('D')  # 'D' for daily frequency, adjust as necessary

# Check again after setting the frequency
print("Adjusted Frequency:", df.index.freq)


Original Frequency: <Day>
Adjusted Frequency: <Day>


In [29]:
df

,Day_ahead_price (€/MWh),Solar_radiation (W/m2),Wind_speed (m/s),Temperature (°C),Biomass (GWh),Hard_coal (GWh),Hydro (GWh),Lignite (GWh),Natural_gas (GWh),Other (GWh),Pumped_storage_generation (GWh),Solar_energy (GWh),Wind_offshore (GWh),Wind_onshore (GWh),Net_total_export_import (GWh),BEV_vehicles,Oil_price (EUR),TTF_gas_price (€/MWh),Nuclear_energy (GWh)
Date,,,,,,,,,,,,,,,,,,,
2012-01-01,18.19,14.75,4.95,8.39,98.605,108.454,51.011,325.337,188.811,54.040,19.314,6.263,3.404,235.467,54.662,6,99.64,21.1000,250.979
2012-01-02,33.82,15.12,5.00,7.41,98.605,222.656,51.862,343.168,229.293,54.166,28.892,6.312,3.350,231.772,-64.477,6,100.04,20.0000,258.671
2012-01-03,35.03,31.88,7.77,5.23,98.605,162.204,48.851,336.773,241.297,53.518,21.072,24.226,7.292,504.484,-35.078,6,100.44,20.9000,271.495
2012-01-04,32.16,25.21,8.04,4.78,98.605,189.633,47.101,323.976,252.289,52.194,28.300,14.157,7.828,541.528,22.924,6,103.15,21.4000,270.613
2012-01-05,20.35,13.46,9.98,4.23,98.605,175.733,45.854,327.502,259.018,52.179,31.887,4.728,8.280,572.819,35.618,6,103.92,21.3000,287.555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-25,61.09,97.58,3.43,4.52,123.683,48.501,58.671,193.697,255.124,60.627,13.565,169.216,29.879,278.787,-36.930,947,75.22,23.7625,0.000
2024-02-26,66.27,73.25,3.12,4.96,124.810,69.146,58.444,281.177,289.764,59.810,12.231,110.504,62.336,239.555,-198.686,947,75.09,23.9000,0.000
2024-02-27,73.84,58.12,3.11,4.53,124.989,103.379,59.181,351.355,354.042,67.170,23.753,85.584,16.951,131.761,-209.332,947,76.11,24.8300,0.000


## Approach with PandasDataset

### Creating a GluonTs data frame

In [30]:
# create an item_id column with the same value for all rows
df['item_id'] = 0

df.columns

Index(['Day_ahead_price (€/MWh)', 'Solar_radiation (W/m2)', 'Wind_speed (m/s)',
       'Temperature (°C)', 'Biomass (GWh)', 'Hard_coal (GWh)', 'Hydro (GWh)',
       'Lignite (GWh)', 'Natural_gas (GWh)', 'Other (GWh)',
       'Pumped_storage_generation (GWh)', 'Solar_energy (GWh)',
       'Wind_offshore (GWh)', 'Wind_onshore (GWh)',
       'Net_total_export_import (GWh)', 'BEV_vehicles', 'Oil_price (EUR)',
       'TTF_gas_price (€/MWh)', 'Nuclear_energy (GWh)', 'item_id'],
      dtype='object')

In [31]:
ds = PandasDataset.from_long_dataframe(df, 
    freq='D', 
    target = 'Day_ahead_price (€/MWh)',
    item_id='item_id',
    feat_dynamic_real=['Solar_radiation (W/m2)', 'Wind_speed (m/s)',
       'Temperature (°C)', 'Biomass (GWh)', 'Hard_coal (GWh)', 'Hydro (GWh)',
       'Lignite (GWh)', 'Natural_gas (GWh)', 'Other (GWh)',
       'Pumped_storage_generation (GWh)', 'Solar_energy (GWh)',
       'Wind_offshore (GWh)', 'Wind_onshore (GWh)',
       'Net_total_export_import (GWh)', 'BEV_vehicles', 'Oil_price (EUR)',
       'TTF_gas_price (€/MWh)', 'Nuclear_energy (GWh)'])

In [32]:
ds

PandasDataset<size=1, freq=D, num_feat_dynamic_real=18, num_past_feat_dynamic_real=0, num_feat_static_real=0, num_feat_static_cat=0, static_cardinalities=[]>

In [33]:
import torch
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
import plotly.graph_objs as go

# Constants
SIZE = "base"  # model size: choose from {'small', 'base', 'large'}
PDT = 790  # prediction length: any positive integer
CTX = 200  # context length: any positive integer
PSZ = "auto"  # patch size: choose from {"auto", 8, 16, 32, 64, 128}
BSZ = 32  # batch size: any positive integer
TEST = 790  # test set length: any positive integer

# Split into train/test set
train, test_template = split(
    ds, offset=-TEST
)  # assign last TEST time steps as test set

# Construct rolling window evaluation
test_data = test_template.generate_instances(
    prediction_length=PDT,  # number of time steps for each prediction
    windows=TEST // PDT,  # number of windows in rolling window evaluation
    distance=PDT,  # number of time steps between each window - distance=PDT for non-overlapping windows
)

# Prepare pre-trained model by downloading model weights from huggingface hub
model = MoiraiForecast(
    module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.0-R-{SIZE}"),
    prediction_length=PDT,
    context_length=CTX,
    patch_size=PSZ,
    num_samples=30,
    target_dim=1,
    feat_dynamic_real_dim=ds.num_feat_dynamic_real,
    past_feat_dynamic_real_dim=ds.num_past_feat_dynamic_real,
)

predictor = model.create_predictor(batch_size=BSZ)
forecasts = predictor.predict(test_data.input)

# Prepare data for Plotly plotting
input_it = iter(test_data.input)
label_it = iter(test_data.label)
forecast_it = iter(forecasts)

inp = next(input_it)
label = next(label_it)
forecast = next(forecast_it)

# Convert tensors to NumPy arrays if needed
inp_values = inp['target'].numpy() if torch.is_tensor(inp['target']) else inp['target']
label_values = label['target'].numpy() if torch.is_tensor(label['target']) else label['target']
forecast_values = forecast.mean.numpy() if torch.is_tensor(forecast.mean) else forecast.mean


In [34]:
# Convert 'start' to a timestamp if necessary
start_date = inp['start'].to_timestamp() if hasattr(inp['start'], 'to_timestamp') else pd.Timestamp(inp['start'])

# Generate test period dates
test_start_date = pd.Timestamp(start_date) + pd.Timedelta(days=len(inp_values))
test_dates = pd.date_range(start=test_start_date, periods=PDT, freq='D')

# Confidence interval traces with reduced dominance
confidence_trace_upper = go.Scatter(
    x=test_dates,
    y=forecast.quantile(0.9).numpy() if torch.is_tensor(forecast.quantile(0.9)) else forecast.quantile(0.9),
    mode='lines',
    name='Confidence Interval Upper',
    line=dict(color='grey', width=1, dash='dot'),
    fill='tonexty',
    fillcolor='rgba(169, 169, 169, 0.2)',  # Light grey fill with reduced opacity
    showlegend=False
)

confidence_trace_lower = go.Scatter(
    x=test_dates,
    y=forecast.quantile(0.1).numpy() if torch.is_tensor(forecast.quantile(0.1)) else forecast.quantile(0.1),
    mode='lines',
    name='Confidence Interval Lower',
    line=dict(color='grey', width=1, dash='dot'),
    fill='tonexty',
    fillcolor='rgba(169, 169, 169, 0.2)',  # Light grey fill with reduced opacity
    showlegend=False
)

# Plotting with Plotly (focusing on the test period)
forecast_trace = go.Scatter(
    x=test_dates,
    y=forecast_values,
    mode='lines',
    name='Forecasted Data',
    line=dict(color='darkred', dash='solid')
)

label_trace = go.Scatter(
    x=test_dates,
    y=label_values,
    mode='lines',
    name='Actual Test Data',
    line=dict(color='darkgreen', dash='solid')
)

# Update layout for better visualization
layout = go.Layout(
    title=f"Test Period Data: Forecast vs Actual (Prediction Length: {PDT}, Context Length: {CTX})",
    xaxis=dict(title='Date', range=[test_dates[0], test_dates[-1]]),  # Set x-axis range to focus on the test period
    yaxis=dict(title='Value'),
    legend=dict(x=0.01, y=0.99),
    template='plotly_white',
)

# Ensure the actual and forecasted data are added last to bring them to the front
fig = go.Figure(data=[confidence_trace_lower, confidence_trace_upper, forecast_trace, label_trace], layout=layout)
fig.show()

## Approach for testing multiple parameter combinations

In [35]:
# Define the length of the test set 
TEST = 790  # test set length: any positive integer
PDT = 790

# Re-run the model with the best parameters
train, test_template = split(
    ds, offset=-TEST
)  # assign last TEST time steps as test set

# Ensure windows is an integer
windows = max(1, int(TEST // PDT))

test_data = test_template.generate_instances(
    prediction_length=PDT,
    windows=windows,
    distance=PDT,
)


In [47]:
import torch
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
import plotly.graph_objs as go
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed  # Import for parallelization

# Constants
SIZE = "base"  # model size: choose from {'small', 'base', 'large'}
PSZ = "auto"  # patch size: choose from {"auto", 8, 16, 32, 64, 128}

# List of parameter combinations to try
PDT = 79  # Use a smaller test data length for prediction length
CTX_list = [200]  # Context lengths to try
BSZ_list = [32]  # Batch sizes to try
NUM_SAMPLES_list = [100]  # Different num_samples to try

# Ensure ds is defined correctly
# Assuming ds is the dataset loaded as PandasDataset
TEST = 790  # Set TEST length to a known constant or compute dynamically based on the dataset

# Function to run a single combination
def run_model(CTX, BSZ, num_samples):
    print(f"Training with CTX={CTX}, BSZ={BSZ}, num_samples={num_samples}")

    # Split into train/test set
    train, test_template = split(ds, offset=-TEST)  # Assign last TEST time steps as test set

    # Ensure test length
    TEST_LEN = len(test_template.data) if hasattr(test_template, 'data') else TEST  # Safe length calculation

    # Compute the number of windows and ensure it's an integer
    windows = max(1, TEST_LEN // PDT)  # Number of windows for the entire test set

    # Construct rolling window evaluation
    test_data = test_template.generate_instances(
        prediction_length=PDT,  # Prediction length for the entire test data
        windows=windows,  # Number of windows for the entire test set
        distance=PDT,  # Full distance to avoid overlapping
    )

    # Prepare pre-trained model by downloading model weights from huggingface hub
    model = MoiraiForecast(
        module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.0-R-{SIZE}"),
        prediction_length=PDT,
        context_length=CTX,
        patch_size=PSZ,
        num_samples=num_samples,  # Use varying num_samples
        target_dim=1,
        feat_dynamic_real_dim=ds.num_feat_dynamic_real,
        past_feat_dynamic_real_dim=ds.num_past_feat_dynamic_real,
    )

    predictor = model.create_predictor(batch_size=BSZ)
    forecasts = predictor.predict(test_data.input)

    # Prepare data for Plotly plotting
    input_it = iter(test_data.input)
    label_it = iter(test_data.label)
    forecast_it = iter(forecasts)

    inp = next(input_it)
    label = next(label_it)
    forecast = next(forecast_it)

    # Convert tensors to NumPy arrays if needed
    inp_values = inp['target'].numpy() if torch.is_tensor(inp['target']) else inp['target']
    label_values = label['target'].numpy() if torch.is_tensor(label['target']) else label['target']
    forecast_values = forecast.mean.numpy() if torch.is_tensor(forecast.mean) else forecast.mean

    # Compute error metrics
    mse = mean_squared_error(label_values, forecast_values)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(label_values, forecast_values)

    return (CTX, BSZ, num_samples, rmse, mse, mae)

# Parallel execution using ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=1) as executor:
    futures = [
        executor.submit(run_model, CTX, BSZ, num_samples)
        for CTX in CTX_list
        for BSZ in BSZ_list
        for num_samples in NUM_SAMPLES_list
    ]

    # Collect results as they complete
    for future in as_completed(futures):
        CTX, BSZ, num_samples, rmse, mse, mae = future.result()
        results_df.loc[len(results_df)] = [CTX, BSZ, num_samples, rmse, mse, mae]

# Find the best combination with the lowest RMSE and MSE
best_combination = results_df.sort_values(by=['RMSE', 'MSE']).iloc[0]
best_CTX = int(best_combination['CTX'])
best_BSZ = int(best_combination['BSZ'])
best_num_samples = int(best_combination['num_samples'])

print(results_df)
print(f"Best combination with least RMSE and MSE: CTX={best_CTX}, BSZ={best_BSZ}, num_samples={best_num_samples}")



Training with CTX=200, BSZ=32, num_samples=100
     CTX   BSZ  num_samples        RMSE           MSE        MAE
0  200.0  32.0        100.0   38.134323   1454.226628  29.663998
1  200.0  32.0        100.0  108.713772  11818.684216  85.419623
Best combination with least RMSE and MSE: CTX=200, BSZ=32, num_samples=100


In [49]:
# Re-run the model with the best parameters
train, test_template = split(
    ds, offset=-TEST
)  # assign last TEST time steps as test set

# Ensure windows is an integer
windows = max(1, int(TEST // PDT))

test_data = test_template.generate_instances(
    prediction_length=PDT,
    windows=windows,
    distance=PDT,
)

model = MoiraiForecast(
    module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.0-R-{SIZE}"),
    prediction_length=PDT,
    context_length=best_CTX,
    patch_size=PSZ,
    num_samples=100,
    target_dim=1,
    feat_dynamic_real_dim=ds.num_feat_dynamic_real,
    past_feat_dynamic_real_dim=ds.num_past_feat_dynamic_real,
)

predictor = model.create_predictor(batch_size=best_BSZ)
forecasts = predictor.predict(test_data.input)

input_it = iter(test_data.input)
label_it = iter(test_data.label)
forecast_it = iter(forecasts)

inp = next(input_it)
label = next(label_it)
forecast = next(forecast_it)

inp_values = inp['target'].numpy() if torch.is_tensor(inp['target']) else inp['target']
label_values = label['target'].numpy() if torch.is_tensor(label['target']) else label['target']
forecast_values = forecast.mean.numpy() if torch.is_tensor(forecast.mean) else forecast.mean

start_date = inp['start'].to_timestamp() if hasattr(inp['start'], 'to_timestamp') else pd.Timestamp(inp['start'])

test_start_date = pd.Timestamp(start_date) + pd.Timedelta(days=len(inp_values))
test_dates = pd.date_range(start=test_start_date, periods=PDT, freq='D')

confidence_trace_upper = go.Scatter(
    x=test_dates,
    y=forecast.quantile(0.9).numpy() if torch.is_tensor(forecast.quantile(0.9)) else forecast.quantile(0.9),
    mode='lines',
    name='Confidence Interval Upper',
    line=dict(color='grey', width=1, dash='dot'),
    fill='tonexty',
    fillcolor='rgba(169, 169, 169, 0.2)',  # Light grey fill with reduced opacity
    showlegend=False
)

confidence_trace_lower = go.Scatter(
    x=test_dates,
    y=forecast.quantile(0.1).numpy() if torch.is_tensor(forecast.quantile(0.1)) else forecast.quantile(0.1),
    mode='lines',
    name='Confidence Interval Lower',
    line=dict(color='grey', width=1, dash='dot'),
    fill='tonexty',
    fillcolor='rgba(169, 169, 169, 0.2)',  # Light grey fill with reduced opacity
    showlegend=False
)

forecast_trace = go.Scatter(
    x=test_dates,
    y=forecast_values,
    mode='lines',
    name='Forecasted Data',
    line=dict(color='darkred', dash='solid')
)

label_trace = go.Scatter(
    x=test_dates,
    y=label_values,
    mode='lines',
    name='Actual Test Data',
    line=dict(color='darkgreen', dash='solid')
)

layout = go.Layout(
    title=f"Best Model: Forecast vs Actual (PDT: {PDT}, CTX: {best_CTX}, BSZ: {best_BSZ})",
    xaxis=dict(title='Date', range=[test_dates[0], test_dates[-1]]),  # Set x-axis range to focus on the test period
    yaxis=dict(title='Value'),
    legend=dict(x=0.01, y=0.99),
    template='plotly_white',
)

fig = go.Figure(data=[confidence_trace_lower, confidence_trace_upper, forecast_trace, label_trace], layout=layout)
fig.show()
